In [1]:
from tkinter import *
import customtkinter
import pandas as pd
import sqlite3
from CTkListbox import *
import tkinter.messagebox as messagebox
from PIL import Image, ImageTk


In [2]:
df = pd.DataFrame()

In [ ]:
#customtkinter.set_appearance_mode("System")
ctk_window = customtkinter.CTk()
ctk_window.title("LeafCat")
ctk_window.geometry('1150x500')


title = StringVar()
content = StringVar()
writer = StringVar()
hashtag = StringVar()
createDate = StringVar()
likes = 0

search_hashtag = StringVar()

# 함수 정의
   
def createBoard():
    # Check for empty fields
    if not title.get() or not content.get() or not writer.get() or not hashtag.get() or not createDate.get():
        print("Please fill in all fields.")
        return
    try:
        # DB 커넥션
        con = sqlite3.connect('leafCat.db')
        cur = con.cursor()

        sql = 'INSERT INTO board(title, writer, content, hastag, create_date, likes) VALUES(?,?,?,?,?,?)'
        con.execute(sql, (title.get(), writer.get(), content.get(), hashtag.get(), createDate.get(), likes))
        con.commit()
        con.close()

        print("Data inserted successfully.")
    except Exception as e:
        print("Error occurred during data insertion:", e)

def updateLikes(selected_data):
    print(selected_data)
    con = sqlite3.connect('leafCat.db')
    cur = con.cursor()
    
    #좋아요 갯수 업데이트
    likes_num = selected_data[6]
    likes_num+=1
    select_sql = "UPDATE board SET likes = ? where board_num = ?"
    cur.execute(select_sql, (likes_num, selected_data[0]))
    
    # 선택한 게시글 데이터 가져오기
    select_sql = "SELECT * FROM board WHERE board_num = ?"
    cur.execute(select_sql, (selected_data[0],))
    data = cur.fetchone()
    print(data) 
    
    showBoard(data[0])
    con.commit()
    con.close()
    
    
def showBoard(selected_board):
    selected_index = selected_board[0]  # 더블클릭한 항목의 인덱스 가져오기
    print(selected_board)
    con = sqlite3.connect('leafCat.db')
    cur = con.cursor()

    # 선택한 게시글 데이터 가져오기
    select_sql = "SELECT * FROM board"
    cur.execute(select_sql)
    data = cur.fetchall()
    selected_data = data[selected_index]
    
    # 새로운 창에 게시글 상세 내역 보여주기
    board_window = customtkinter.CTkToplevel(ctk_window)
    board_window.title(selected_data[1])
    board_window.geometry('200x300')
    
    board_frame = Frame(board_window, width=200, height=300)
    board_frame.grid()
    
    # 게시글 상세 내역 라벨 생성
    title_label = Label(board_frame, text="제목     " + selected_data[1])
    title_label.grid(row=0, column=0, padx=10, pady=10)
    
    writer_label = Label(board_frame, text="작성자     " + selected_data[2])
    writer_label.grid(row=1, column=0, padx=10, pady=10)
    
    content_label = Label(board_frame, text="내용     " + selected_data[3])
    content_label.grid(row=2, column=0, padx=10, pady=10)
    
    hashtag_label = Label(board_frame, text="해시태그     " + selected_data[4])
    hashtag_label.grid(row=3, column=0, padx=10, pady=10)
    
    date_label = Label(board_frame, text="작성일     " + selected_data[5])
    date_label.grid(row=4, column=0, padx=10, pady=10)
    
    likes_label = Label(board_frame, text="좋아요     " + str(selected_data[6]))
    likes_label.grid(row=5, column=0, padx=10, pady=10)
    
    likes_img = PhotoImage(file='images/likes.png')
    likes_btn = customtkinter.CTkButton(master=board_frame, text='', width=60, height=60, image=likes_img)
    likes_btn.grid(row=6, column=0)
    likes_btn.bind('<Button-1>', lambda event: updateLikes(selected_data))
    con.close()
    
def sortByLikes(listbox):
    listbox.delete(0, END)  # 기존 데이터 삭제
    
    con = sqlite3.connect('leafCat.db')
    cur = con.cursor()

    select_sql = "SELECT * FROM board ORDER BY likes DESC"
    cur.execute(select_sql)
    data = cur.fetchall()
    
    df = pd.DataFrame(data)
    df.columns = ['게시글번호','제목','작성자','내용','해시태그','작성일','좋아요']
    df.set_index(keys=['게시글번호'], inplace=True)
    
    for i in range(len(df['작성자'])):
        title_data = df['제목'].values[i]       
        listbox.insert(END, df['제목'].values[i])
    con.close()

    # 더블클릭 이벤트 바인딩
    listbox.bind('<Double-Button-1>', lambda event: showBoard(listbox.curselection()))

def sortByDates(listbox):
    listbox.delete(0, END)  # 기존 데이터 삭제
    
    con = sqlite3.connect('leafCat.db')
    cur = con.cursor()

    select_sql = "SELECT * FROM board ORDER BY create_date ASC"
    cur.execute(select_sql)
    data = cur.fetchall()
    
    df = pd.DataFrame(data)
    df.columns = ['게시글번호','제목','작성자','내용','해시태그','작성일','좋아요']
    df.set_index(keys=['게시글번호'], inplace=True)
    
    for i in range(len(df['작성자'])):
        title_data = df['제목'].values[i]       
        listbox.insert(END, df['제목'].values[i])
    con.close()

    # 더블클릭 이벤트 바인딩
    listbox.bind('<Double-Button-1>', lambda event: showBoard(listbox.curselection()))

def searchByhashTags():
    listbox.delete(0, END)  # 기존 데이터 삭제

    con = sqlite3.connect('leafCat.db')
    cur = con.cursor()

    select_sql = "SELECT * FROM board WHERE hastag = ?"
    cur.execute(select_sql, (search_hashtag.get(),))
    data = cur.fetchall()

    df = pd.DataFrame(data)
    df.columns = ['게시글번호', '제목', '작성자', '내용', '해시태그', '작성일', '좋아요']
    df.set_index(keys=['게시글번호'], inplace=True)

    for i in range(len(df['작성자'])):
        title_data = df['제목'].values[i]
        listbox.insert(END, df['제목'].values[i])

    con.close()

    # 더블클릭 이벤트 바인딩
    listbox.bind('<Double-Button-1>', lambda event: showBoard(listbox.curselection()))

def ctk_entry():
    entry_window = customtkinter.CTkToplevel(ctk_window)
    entry_window.title('New Board')
    entry_window.geometry('300x550')
    board_label = Label(master=entry_window, text='게시글 작성')
    board_label.grid()
    
    frame = Frame(entry_window, width=300, height=550)
    frame.grid()
    
    # 게시글 작성 엔트리    
    input_title = customtkinter.CTkEntry(frame, textvariable=title)
    input_content = customtkinter.CTkEntry(frame, textvariable=content)
    input_writer = customtkinter.CTkEntry(frame, textvariable=writer)    
    input_hashtag = customtkinter.CTkEntry(frame, textvariable=hashtag)
    input_date = customtkinter.CTkEntry(frame, textvariable=createDate)
    
    input_title.grid(row=0, column=0, padx=10, pady=10, ipadx=40, ipady=10)
    input_content.grid(row=1, column=0, padx=10, pady=10, ipadx=40, ipady=10)
    input_writer.grid(row=2, column=0, padx=10, pady=10, ipadx=40, ipady=50)
    input_hashtag.grid(row=3, column=0, padx=10, pady=10, ipadx=40, ipady=10)
    input_date.grid(row=4, column=0, padx=10, pady=10, ipadx=40, ipady=10)
    
    print(title.get())
    create_btn = customtkinter.CTkButton(master=frame, text='Create', command=createBoard)
    create_btn.grid(row=5, column=0, padx=20, pady=20)
    
    entry_window.mainloop()

    
############################ 메인화면 ############################

# 메인 로고


logo_img = PhotoImage(file='images/logo.png')


#검색 엔트리
search_frame = Frame(ctk_window, width=200, height=300)
search_frame.grid(row=0, column=1)

main_logo = customtkinter.CTkLabel(search_frame, text='' ,image=logo_img)
main_logo.grid(row=0, column=1) 

hashtag_label = Label(master=search_frame, text='해시태그 검색')
hashtag_label.grid(row=1, column=1)

hastag_entry = customtkinter.CTkEntry(search_frame, textvariable=search_hashtag)
hastag_entry.grid(row=2, column=1, padx=20, pady=2)


#게시글 리스트바, 스크롤바 
listbox_frame = Frame(ctk_window, width=100, height=20)
listbox_frame.grid(row=0, column=2, columnspan=3)

listbox = Listbox(master=listbox_frame,selectmode='extended', width=100, height=23)
listbox.grid(row=0, column=2, columnspan=3, sticky="nsew")

scroll = customtkinter.CTkScrollbar(ctk_window, command=listbox.yview)
scroll.grid(row=0, column=5, ipadx=3, ipady=100)

listbox.config(yscrollcommand=scroll.set)

# 하단버튼
board_btn = customtkinter.CTkButton(master=ctk_window, text='New', command=lambda: ctk_entry())
likes_order_btn = customtkinter.CTkButton(master=ctk_window, text='좋아요순', command=lambda: sortByLikes(listbox))
dates_order_btn = customtkinter.CTkButton(master=ctk_window, text='날짜순', command=lambda: sortByDates(listbox))
search_hastag_entry = customtkinter.CTkButton(master=ctk_window, text='검색', command=searchByhashTags)

board_btn.grid(row=1, column=1, padx=20, pady=20)
likes_order_btn.grid(row=1, column=2, padx=20, pady=20)
dates_order_btn.grid(row=1, column=3, padx=20, pady=20)
search_hastag_entry.grid(row=1, column=4, padx=20, pady=20)

ctk_window.mainloop()

/Users/jayden/opt/anaconda3/lib/python3.9/site-packages/customtkinter/windows/widgets/core_widget_classes/ctk_base_class.py:179: UserWarning: CTkLabel Warning: Given image is not CTkImage but <class 'tkinter.PhotoImage'>. Image can not be scaled on HighDPI displays, use CTkImage instead.

  warnings.warn(f"{type(self).__name__} Warning: Given image is not CTkImage but {type(image)}. Image can not be scaled on HighDPI displays, use CTkImage instead.\n")


(0,)


/Users/jayden/opt/anaconda3/lib/python3.9/site-packages/customtkinter/windows/widgets/core_widget_classes/ctk_base_class.py:179: UserWarning: CTkButton Warning: Given image is not CTkImage but <class 'tkinter.PhotoImage'>. Image can not be scaled on HighDPI displays, use CTkImage instead.

  warnings.warn(f"{type(self).__name__} Warning: Given image is not CTkImage but {type(image)}. Image can not be scaled on HighDPI displays, use CTkImage instead.\n")


(1,)
(37,)
(37,)
(73, '제목123', '내용123', '김민엽', '#한국공학대', '2023-05-20', 1)
(73, '제목123', '내용123', '김민엽', '#한국공학대', '2023-05-20', 2)


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/jayden/opt/anaconda3/lib/python3.9/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/var/folders/ns/yz1k0nqj0g97_5ts2x6t8y4r0000gn/T/ipykernel_15812/3516671594.py", line 101, in <lambda>
    likes_btn.bind('<Button-1>', lambda event: updateLikes(selected_data))
  File "/var/folders/ns/yz1k0nqj0g97_5ts2x6t8y4r0000gn/T/ipykernel_15812/3516671594.py", line 54, in updateLikes
    showBoard(data[0])
  File "/var/folders/ns/yz1k0nqj0g97_5ts2x6t8y4r0000gn/T/ipykernel_15812/3516671594.py", line 60, in showBoard
    selected_index = selected_board[0]  # 더블클릭한 항목의 인덱스 가져오기
TypeError: 'int' object is not subscriptable


In [ ]:
print(df)

### 